In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from category_encoders import TargetEncoder, one_hot
from catboost import *
%matplotlib inline

In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X13,X14,X15,X16,X17,X18,X19,X20,X21,target
0,0,2,908749,954,480270,935,71,79,53,89,...,51,100,87,3,61,48,43,79,329,0
1,1,1,438444,2162,486685,2154,32,39,48,49,...,39,83,91,6,57,94,15,89,581,1
2,2,1,596915,2066,711059,2081,43,21,58,94,...,7,52,37,50,69,60,89,10,548,0
3,3,1,625198,1508,66810,1474,7,1,34,73,...,39,27,55,97,83,1,88,87,555,1
4,4,2,228654,1202,542816,1196,53,35,16,80,...,35,95,50,55,10,49,12,68,651,0


# Data Prepairing

In [3]:
data.describe()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X13,X14,X15,X16,X17,X18,X19,X20,X21,target
count,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,...,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06
mean,1.816502e+06,1.650469e+00,5.638035e+05,1.449992e+03,5.637574e+05,1.450097e+03,5.016512e+01,5.016514e+01,5.018091e+01,5.015996e+01,...,5.017506e+01,5.017908e+01,5.018681e+01,5.018890e+01,5.014741e+01,5.018218e+01,5.016238e+01,5.015347e+01,4.913625e+02,3.800829e-01
std,1.048758e+06,7.179981e-01,3.248814e+05,4.848098e+02,3.247859e+05,4.848208e+02,2.964439e+01,2.966132e+01,2.963772e+01,2.964519e+01,...,2.965711e+01,2.965469e+01,2.964957e+01,2.965893e+01,2.965769e+01,2.965730e+01,2.963982e+01,2.966294e+01,1.464528e+02,4.854070e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,0.000000e+00
25%,9.082510e+05,1.000000e+00,2.829920e+05,1.074000e+03,2.829370e+05,1.074000e+03,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,...,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,3.840000e+02,0.000000e+00
50%,1.816502e+06,2.000000e+00,5.637460e+05,1.370000e+03,5.639180e+05,1.371000e+03,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,...,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.350000e+02,0.000000e+00
75%,2.724753e+06,2.000000e+00,8.454180e+05,1.747000e+03,8.454590e+05,1.747000e+03,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,...,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,6.150000e+02,1.000000e+00
max,3.633004e+06,8.000000e+00,1.126077e+06,4.186000e+03,1.126076e+06,4.192000e+03,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,...,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,6.800000e+02,1.000000e+00


In [72]:
y = data.target
# X = data.drop(['target', 'id', 'X1', 'X3'], axis=1)
X = data.drop(['target', 'id'], axis=1)

In [84]:
army_count_1 = X['X5'] + X['X7'] + X['X9'] + X['X11'] + X['X13'] + X['X15'] + X['X17'] + X['X19']
army_count_2 = X['X6'] + X['X8'] + X['X10'] + X['X12'] + X['X14'] + X['X16'] + X['X18'] + X['X20']
X['army_relation'] = army_count_1 - army_count_2
X['rank_relation'] = X['X2'] - (X['X4'])

In [85]:
first_player_army = ['X5', 'X7', 'X9', 'X11', 'X13', 'X15', 'X17', 'X19']
second_player_army = ['X6', 'X8', 'X10', 'X12', 'X14', 'X16', 'X18', 'X20']

for i in range(len(first_player_army)):
    unit_1 = first_player_army[i]
    unit_2 = second_player_army[i]
    X[unit_1 + '-' + unit_2] = X[unit_1] - (X[unit_2])

In [86]:
X.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,army_relation,rank_relation,X5-X6,X7-X8,X9-X10,X11-X12,X13-X14,X15-X16,X17-X18,X19-X20
0,2,908749,954,480270,935,71,79,53,89,15,...,-23,19,-8,-36,14,-5,-49,84,13,-36
1,1,438444,2162,486685,2154,32,39,48,49,100,...,35,8,-7,-1,82,31,-44,85,-37,-74
2,1,596915,2066,711059,2081,43,21,58,94,88,...,-34,-15,22,-36,-5,-45,-45,-13,9,79
3,1,625198,1508,66810,1474,7,1,34,73,6,...,-81,34,6,-39,-82,-19,12,-42,82,1
4,2,228654,1202,542816,1196,53,35,16,80,72,...,-214,6,18,-64,-9,1,-60,-5,-39,-56


In [88]:
# cat_features = list(range(0, X.shape[1] - 12))
cat_features = list(range(0, X.shape[1]) )
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [89]:
print('Labels: {}'.format(set(y)))
print('Zero count = {}, One count = {}'.format(len(y) - sum(y), sum(y)))

Labels: {0, 1}
Zero count = 2252162, One count = 1380843


In [90]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=42, shuffle=True)

# Model training

In [79]:
# !rm 'catboost_info/snapshot.bkp'
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=1000,
    verbose=10,
    random_seed=42,
    learning_rate=0.5,
    # save_snapshot=True,
    # snapshot_file='snapshot.bkp',
    # snapshot_interval=1,
    l2_leaf_reg=3.0,
    early_stopping_rounds=20,
    one_hot_max_size=5,
    bootstrap_type='Bernoulli',
    subsample=0.5,
    random_strength=2,
    use_best_model=True,
    best_model_min_trees=30,
    rsm=0.5,
    leaf_estimation_iterations=5
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    plot=False
)

0:	learn: 0.6712609	test: 0.6712265	best: 0.6712265 (0)	total: 6.56s	remaining: 1h 49m 9s
10:	learn: 0.6580620	test: 0.6560370	best: 0.6560370 (10)	total: 1m 40s	remaining: 2h 30m 12s
20:	learn: 0.6560526	test: 0.6544962	best: 0.6544962 (20)	total: 3m 12s	remaining: 2h 29m 20s
30:	learn: 0.6552551	test: 0.6535943	best: 0.6535943 (30)	total: 4m 40s	remaining: 2h 26m 15s
40:	learn: 0.6544921	test: 0.6529765	best: 0.6528328 (39)	total: 6m 13s	remaining: 2h 25m 39s
50:	learn: 0.6541264	test: 0.6526041	best: 0.6526041 (50)	total: 7m 35s	remaining: 2h 21m 14s
60:	learn: 0.6538168	test: 0.6522986	best: 0.6522986 (60)	total: 9m 4s	remaining: 2h 19m 36s
70:	learn: 0.6536462	test: 0.6521964	best: 0.6521964 (70)	total: 10m 15s	remaining: 2h 14m 8s
80:	learn: 0.6534664	test: 0.6520941	best: 0.6520879 (77)	total: 11m 23s	remaining: 2h 9m 18s
90:	learn: 0.6532980	test: 0.6520265	best: 0.6520265 (90)	total: 12m 34s	remaining: 2h 5m 39s
100:	learn: 0.6531473	test: 0.6519592	best: 0.6519592 (100)	total

In [69]:
model.tree_count_

153

# Feature Importance

In [80]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,X3,27.197863
1,X1,26.821290
2,X4,7.131163
3,X5,2.486621
4,rank_relation,2.409170
5,X18,2.405036
6,X10,2.287759
7,X9,2.126740
8,X16,2.123085
9,X19,2.121300


# Hyperparameter tunning

# Saving Model

# Calculate predictions for the contest

In [21]:
data_test = pd.read_csv('test.csv')
data_test.head()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21
0,0,1,1032217,1187,1011523,1172,57,100,1,80,...,79,45,18,35,10,80,81,16,0,547
1,1,2,1059033,2128,505263,2104,82,27,87,79,...,74,96,55,54,48,3,92,5,51,656
2,2,1,207787,1017,822639,1040,82,48,43,62,...,16,64,35,70,99,60,30,37,45,437
3,3,1,1095582,1449,32841,1424,71,43,11,98,...,1,7,41,94,26,52,45,81,1,589
4,4,2,963764,1202,489871,1182,11,62,16,55,...,26,60,37,7,1,80,48,50,52,340


In [22]:
X_test = data_test.drop(['id', 'X1', 'X3'], axis=1)
X_test.head()

,X0,X2,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21
0,1,1187,1172,57,100,1,80,25,87,43,79,45,18,35,10,80,81,16,0,547
1,2,2128,2104,82,27,87,79,80,71,88,74,96,55,54,48,3,92,5,51,656
2,1,1017,1040,82,48,43,62,18,60,83,16,64,35,70,99,60,30,37,45,437
3,1,1449,1424,71,43,11,98,80,33,35,1,7,41,94,26,52,45,81,1,589
4,2,1202,1182,11,62,16,55,88,92,76,26,60,37,7,1,80,48,50,52,340


In [23]:
y_test = model.predict_proba(X_test)
y_test = y_test[:, 1]

# Prepare the submission

In [24]:
ans = pd.DataFrame(data=y_test, columns=["target"])
ans.reset_index(level=0, inplace=True)
ans.reset_index(drop=True, inplace=True)
ans = ans.rename(columns={'index':'id'})
ans.head()

,id,target
0,0,0.392236
1,1,0.363315
2,2,0.371697
3,3,0.346989
4,4,0.307991


In [25]:
ans.to_csv('submission.csv',index=False)